In [15]:
from pathlib import Path
import torch
from ray.rllib.models.torch.torch_distributions import TorchDiagGaussian
from ray.tune.result import TRAINING_ITERATION
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from ray.rllib.core import (
    COMPONENT_LEARNER,
    COMPONENT_LEARNER_GROUP,
    COMPONENT_RL_MODULE,
)
from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.examples.envs.classes.multi_agent import MultiAgentPendulum
from ray.rllib.utils.metrics import (
    ENV_RUNNER_RESULTS,
    EPISODE_RETURN_MEAN,
    NUM_ENV_STEPS_SAMPLED_LIFETIME,
)
from ray.rllib.utils.numpy import convert_to_numpy
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    check,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env

from pettingzoo.sisl import waterworld_v4

from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.core.rl_module.multi_rl_module import MultiRLModuleSpec
from ray.rllib.core.rl_module.rl_module import RLModuleSpec
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env
import os

import gymnasium as gym
import numpy as np
import tree  # pip install dm_tree

from ray.rllib.core import DEFAULT_MODULE_ID
from ray.rllib.core.columns import Columns
from ray.rllib.core.rl_module.rl_module import RLModule
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.numpy import convert_to_numpy, softmax
from ray.rllib.utils.metrics import (
    ENV_RUNNER_RESULTS,
    EPISODE_RETURN_MEAN,
)

torch, _ = try_import_torch()

In [16]:
import wandb
# from ray.rllib.algorithms.callbacks import DefaultCallbacks

# class WandbLoggingCallback(DefaultCallbacks):
#     def on_train_result(self, *, algorithm, result, **kwargs):
#         # 提取 episode reward env_runners
#         print(f"[DEBUG] result keys: {list(result.keys())}")
#         print(f"[DEBUG] result keys: {list(result['env_runners'])}")

#         print("[DEBUG] on_train_result 被调用")
#         mean_reward = result.get("episode_reward_mean") or result.get("hist_stats", {}).get("episode_reward_mean")

#         if mean_reward is not None:
#             # Log 到 wandb
#             wandb.log({"episode_reward_mean": mean_reward, "training_iteration": result.get("training_iteration", 0)})

In [17]:
# 设置参数
import sys
sys.argv = [
    'notebook_script.py',
    '--enable-new-api-stack',
    '--num-agents=4',
    # 新增参数用于指定predator和prey的数量
    '--n-predators=2',
    '--wandb-key=fdd7656f474bba144dea1887bcdab534bc7df647',
    '--wandb-project=waterworld-v4',
    '--n-preys=2', 
    '--checkpoint-at-end',
    '--stop-reward=200.0',
    '--checkpoint-freq=1',
]

In [ ]:
parser = add_rllib_example_script_args(
    default_iters=2,
    default_timesteps=10000,
    default_reward=0.0,
)

In [19]:
# 添加新的参数解析
parser.add_argument(
    "--n-predators",
    type=int,
    default=2,
    help="Number of predator agents"
)
parser.add_argument(
    "--n-preys", 
    type=int,
    default=2,
    help="Number of prey agents"
)
parser.add_argument(
    "--use-onnx-for-inference",
    action="store_true",
    help="Whether to convert the loaded module to ONNX format and then perform "
    "inference through this ONNX model.",
)
parser.add_argument(
    "--explore-during-inference",
    action="store_true",
    help="Whether the trained policy should use exploration during action "
    "inference.",
)
parser.add_argument(
    "--num-episodes-during-inference",
    type=int,
    default=10,
    help="Number of episodes to do inference over (after restoring from a checkpoint).",
)

_StoreAction(option_strings=['--num-episodes-during-inference'], dest='num_episodes_during_inference', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, required=False, help='Number of episodes to do inference over (after restoring from a checkpoint).', metavar=None)

In [20]:

args = parser.parse_args()

# 验证参数
assert args.n_predators > 0, "Must set --n-predators > 0 when running this script!"
assert args.n_preys > 0, "Must set --n-preys > 0 when running this script!"
assert (
    args.enable_new_api_stack
), "Must set --enable-new-api-stack when running this script!"

# 计算总智能体数量
total_agents = args.n_predators + args.n_preys
print(f"参数解析完成: n_predators={args.n_predators}, n_preys={args.n_preys}, total_agents={total_agents}, algo={args.algo}")


参数解析完成: n_predators=2, n_preys=2, total_agents=4, algo=PPO


In [21]:
import wandb

wandb.init(project=args.wandb_project, config=vars(args))  # 放在main训练逻辑之前

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


In [22]:
agent_algos = ["PPO", "PPO", "DQN", "DQN", "A2C"]
# from pettingzoo.sisl import waterworld_v4

# agent_algos = ["PPO", "PPO", "DQN", "DQN", "A2C"]

# env = waterworld_v4.env(
#     render_mode="human",
#     n_predators=2,
#     n_preys=3,
#     n_evaders=5,
#     n_obstacles=1,
#     n_poisons=1,
#     agent_algorithms=agent_algos
# )
# 修改环境注册，传递predator和prey的数量
register_env("env", lambda _: PettingZooEnv(
    waterworld_v4.env(
        n_predators=args.n_predators,
        n_preys=args.n_preys,  # 注意：这里应该是n_preys而不是n_prey
        agent_algorithms=agent_algos
    )
))


In [23]:

# 创建新的policies字典，匹配环境中的agent命名
predator_policies = [f"predator_{i}" for i in range(args.n_predators)]
prey_policies = [f"prey_{i}" for i in range(args.n_preys)]
all_policies = predator_policies + prey_policies
print(all_policies)

['predator_0', 'predator_1', 'prey_0', 'prey_1']


In [24]:
# 创建RL module specs字典
rl_module_specs = {p: RLModuleSpec() for p in all_policies}
print(f"创建的policies: {list(all_policies)}")
print(f"创建的RL module specs: {list(rl_module_specs.keys())}")

创建的policies: ['predator_0', 'predator_1', 'prey_0', 'prey_1']
创建的RL module specs: ['predator_0', 'predator_1', 'prey_0', 'prey_1']


In [25]:
base_config = (
    get_trainable_cls(args.algo)
    .get_default_config()
    .environment("env")
    .multi_agent(
        # 在新API中，只需要指定policy_mapping_fn
        policies=set(all_policies),
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .training(
        vf_loss_coeff=0.005,
    )
    .rl_module(
        rl_module_spec=MultiRLModuleSpec(
            rl_module_specs=rl_module_specs,
        ),
        model_config=DefaultModelConfig(vf_share_layers=True),
    )
    # .callbacks(WandbLoggingCallback)  # ✅ 加入回调
)


In [26]:

# 训练
print("开始训练...")
run_rllib_example_script_experiment(base_config, args, keep_ray_up=True)
print("训练完成")

2025-07-11 11:43:26,951	INFO worker.py:1747 -- Calling ray.init() again after it has already been called.


开始训练...
== Status ==
Current time: 2025-07-11 11:43:27 (running for 00:00:00.11)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-07-10_21-32-26_038975_2057396/artifacts/2025-07-11_11-43-26/PPO_2025-07-11_11-43-26/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-07-11 11:43:32 (running for 00:00:05.13)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-07-10_21-32-26_038975_2057396/artifacts/2025-07-11_11-43-26/PPO_2025-07-11_11-43-26/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-07-11 11:43:37 (running for 00:00:10.18)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-07-10_21-32-26_038975_2057396/artifacts/2025-07-11_11-43-2

2025-07-11 11:43:39,226	WARNING util.py:201 -- The `callbacks.on_trial_result` operation took 1.626 s, which may be a performance bottleneck.
2025-07-11 11:43:39,227	WARNING util.py:201 -- The `process_trial_result` operation took 1.628 s, which may be a performance bottleneck.
2025-07-11 11:43:39,228	WARNING util.py:201 -- Processing trial results took 1.629 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2025-07-11 11:43:39,229	WARNING util.py:201 -- The `process_trial_result` operation took 1.630 s, which may be a performance bottleneck.


== Status ==
Current time: 2025-07-11 11:43:42 (running for 00:00:15.21)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-07-10_21-32-26_038975_2057396/artifacts/2025-07-11_11-43-26/PPO_2025-07-11_11-43-26/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2025-07-11 11:43:47 (running for 00:00:20.29)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-07-10_21-32-26_038975_2057396/artifacts/2025-07-11_11-43-26/PPO_2025-07-11_11-43-26/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2025-07-11 11:43:52 (running for 00:00:25.37)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-07-10_21-32-26_038975_2057396/artifacts/2025-07-11_11-43-26/PPO_20

2025-07-11 12:12:32,325	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/qrbao/ray_results/PPO_2025-07-11_11-43-26' in 0.1012s.


== Status ==
Current time: 2025-07-11 12:12:32 (running for 00:29:05.37)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-07-10_21-32-26_038975_2057396/artifacts/2025-07-11_11-43-26/PPO_2025-07-11_11-43-26/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+---------------------+------------+----------------------+--------+------------------+--------+-------------------+-----------------+---------------------+-----------------+---------------------+
| Trial name          | status     | loc                  |   iter |   total time (s) |     ts |   combined return |   return prey_1 |   return predator_1 |   return prey_0 |   return predator_0 |
|---------------------+------------+----------------------+--------+------------------+--------+-------------------+-----------------+---------------------+-----------------+---------------------|
| PPO_env_d249c_00000 | TERMINATED | 192.168.0.25:2

2025-07-11 12:12:36,227	INFO tune.py:1041 -- Total run time: 1749.27 seconds (1745.27 seconds for the tuning loop).


训练完成


In [27]:

# # 训练
# print("开始训练...")
# results = run_rllib_example_script_experiment(base_config, args, keep_ray_up=True)
# print("训练完成")

In [28]:
# 获取最佳结果
print("获取最佳checkpoint...")
best_result = results.get_best_result(
    metric=f"{ENV_RUNNER_RESULTS}/{EPISODE_RETURN_MEAN}", mode="max"
)

获取最佳checkpoint...


NameError: name 'results' is not defined

In [ ]:
print("加载所有智能体的RLModule...")
rl_modules = {}
# 修改：使用新的智能体命名方式
predator_agents = [f"predator_{i}" for i in range(args.n_predators)]
prey_agents = [f"prey_{i}" for i in range(args.n_preys)]
all_agent_names = predator_agents + prey_agents


In [ ]:

for agent_name in all_agent_names:
    rl_module_path = os.path.join(
        best_result.checkpoint.path,
        "learner_group",
        "learner",
        "rl_module",
        agent_name,
    )
    
    if os.path.exists(rl_module_path):
        rl_modules[agent_name] = RLModule.from_checkpoint(rl_module_path)
        print(f"成功加载 {agent_name} 的模型")
    else:
        print(f"警告: 找不到 {agent_name} 的模型路径: {rl_module_path}")
print(f"总共加载了 {len(rl_modules)} 个智能体模型")

In [ ]:

# 推理阶段
print("开始推理...")

def get_action_from_rl_module(rl_module, observation, explore=False):
    """从RLModule获取动作"""
    # 将观察转换为torch tensor并添加batch维度
    input_dict = {Columns.OBS: torch.from_numpy(observation).unsqueeze(0)}
    
    if explore:
        # 使用探索性前向传播
        rl_module_out = rl_module.forward_exploration(input_dict)
        action_dist_inputs = rl_module_out["action_dist_inputs"][0]
        action = TorchDiagGaussian.from_logits(action_dist_inputs.unsqueeze(0)).sample().squeeze(0).numpy()
    else:
        # 使用推理前向传播
        rl_module_out = rl_module.forward_inference(input_dict)
        action_dist_inputs = rl_module_out["action_dist_inputs"][0]
        action = TorchDiagGaussian.from_logits(action_dist_inputs.unsqueeze(0)).sample().squeeze(0).numpy()
    
    return action

In [ ]:

# 运行推理episodes
for episode in range(args.num_episodes_during_inference):
    print(f"\n=== Episode {episode + 1} ===")
    
    # 创建环境（使用新的参数方式）
    env = waterworld_v4.env(
        render_mode="human",
        n_predators=args.n_predators,  # 修改：使用新的参数
        n_preys=args.n_preys,         # 修改：使用新的参数
        n_evaders=5,  # 可以根据需要调整
        n_poisons=10,  # 可以根据需要调整
    )
    env.reset(seed=42 + episode)
    
    episode_rewards = {agent: 0 for agent in env.agents}
    step_count = 0
    
    try:
        for agent in env.agent_iter():
            observation, reward, termination, truncation, info = env.last()
            
            # 累积奖励
            if agent in episode_rewards:
                episode_rewards[agent] += reward
            
            if termination or truncation:
                action = None
                print(f"{agent} 终止, 奖励: {reward}")
            else:
                # 使用对应的RLModule获取动作
                if agent in rl_modules:
                    try:
                        action = get_action_from_rl_module(
                            rl_modules[agent], 
                            observation, 
                            explore=args.explore_during_inference
                        )
                        print(f"{agent} 执行动作: {action}")
                    except Exception as e:
                        print(f"为 {agent} 获取动作时出错: {e}")
                        # 如果出错，使用随机动作作为备选
                        action = env.action_space(agent).sample()
                        print(f"{agent} 使用随机动作: {action}")
                else:
                    # 如果没有找到对应的模型，使用随机动作
                    action = env.action_space(agent).sample()
                    print(f"{agent} 没有找到对应模型，使用随机动作: {action}")
            
            env.step(action)
            step_count += 1
            
            # 每100步输出一次进度
            if step_count % 100 == 0:
                print(f"已执行 {step_count} 步")
                
    except KeyboardInterrupt:
        print("用户中断")
        break
    except Exception as e:
        print(f"Episode {episode + 1} 出现错误: {e}")
    finally:
        env.close()
    
    # 输出episode结果
    print(f"Episode {episode + 1} 完成")
    print("各智能体总奖励:")
    for agent, total_reward in episode_rewards.items():
        print(f"  {agent}: {total_reward:.2f}")
    print(f"总步数: {step_count}")

print("推理完成!")

In [ ]:
# 推理完每个 episode 后记录 reward
wandb.log({
    "episode_index": episode,
    "total_reward_mean": np.mean(list(episode_rewards.values())),
    "total_reward_std": np.std(list(episode_rewards.values())),
})

In [ ]:
# from pettingzoo.sisl import waterworld_v4

# env = waterworld_v4.env(render_mode="human")
# env.reset(seed=42)

# for agent in env.agent_iter():
#     observation, reward, termination, truncation, info = env.last()

#     if termination or truncation:
#         action = None
#     else:
#         # this is where you would insert your policy
#         action = env.action_space(agent).sample()

#     env.step(action)
# env.close()